In [ ]:
!sudo apt-get install openjdk-8-jdk
!java -version
!pip install --ignore-installed -q pyspark==2.4.4
!pip install spark-nlp

In [3]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
#import svgwrite

In [4]:
spark = sparknlp.start()

In [5]:

MODEL_NAME='dependency_typed_conllu'

In [6]:
## Generating Example Files ##
text_list = [
             """John Snow is a good man. He knows a lot about science.""",
             """In what country is the WTO headquartered?""",
             """I was wearing my dark blue shirt and tie.""",
             """The Geneva Motor Show is the most popular car show of the year.""",
             """Bill Gates and Steve Jobs had periods of civility.""",
]


In [ ]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pos = PerceptronModel.pretrained("pos_anc", 'en')\
        .setInputCols("document", "token")\
        .setOutputCol("pos")

dep_parser = DependencyParserModel.pretrained('dependency_conllu')\
        .setInputCols(["document", "pos", "token"])\
        .setOutputCol("dependency")


typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu')\
        .setInputCols(["token", "pos", "dependency"])\
        .setOutputCol("dependency_type")


nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          pos,
          chunker,
          dep_parser,
          typed_dep_parser
      ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [15]:
index=0
df = spark.createDataFrame(pd.DataFrame({"text":[text_list[index]]}))
result = pipelineModel.transform(df)

In [ ]:

result.select(F.explode(F.arrays_zip('token.result',
                                     'token.begin',
                                     'token.end', 
                                     'pos.result', 
                                     'dependency.result', 
                                                  'dependency.metadata',
                                                  'dependency_type.result')).alias("cols"))\
                                                  .select(F.expr("cols['0']").alias("chunk"),
                                                          F.expr("cols['1']").alias("begin"),
                                                          F.expr("cols['2']").alias("end"),
                                                          F.expr("cols['3']").alias("pos"),
                                                          F.expr("cols['4']").alias("dependency"),
                                                          F.expr("cols['5']").alias("dependency_start"),
                                                          F.expr("cols['6']").alias("dependency_type")).toPandas()
